In [ ]:
from os import listdir, path
import re

import time
import numpy as np
import random
import math

In [ ]:
BASE_DIR_PATH = path.dirname(path.abspath("__file__"))  # gdyby nie działało, usuń cudzysłów
INSTANCES_PATH = path.join(BASE_DIR_PATH, "instances")
NEGATIVE_RANDOM_PATH = path.join(INSTANCES_PATH, "negative_random")
NEGATIVE_REPETITIONS_PATH = path.join(INSTANCES_PATH, "negative_repetitions")
POSITIVE_END_ERRORS_PATH = path.join(INSTANCES_PATH, "positive_end_errors")
POSITIVE_RANDOM_PATH = path.join(INSTANCES_PATH, "positive_random")

In [ ]:
negative_random_filenames = [path.join(NEGATIVE_RANDOM_PATH, f) for f in listdir(NEGATIVE_RANDOM_PATH) if path.isfile(path.join(NEGATIVE_RANDOM_PATH, f))]
negative_repetitions_filenames = [path.join(NEGATIVE_REPETITIONS_PATH, f) for f in listdir(NEGATIVE_REPETITIONS_PATH) if path.isfile(path.join(NEGATIVE_REPETITIONS_PATH, f))]
positive_end_errors_filenames = [path.join(POSITIVE_END_ERRORS_PATH, f) for f in listdir(POSITIVE_END_ERRORS_PATH) if path.isfile(path.join(POSITIVE_END_ERRORS_PATH, f))]
positive_random_filenames = [path.join(POSITIVE_RANDOM_PATH, f) for f in listdir(POSITIVE_RANDOM_PATH) if path.isfile(path.join(POSITIVE_RANDOM_PATH, f))]

selected_filenames = negative_random_filenames + negative_repetitions_filenames + positive_end_errors_filenames + positive_random_filenames

In [ ]:
for filename in selected_filenames:
    with open(filename, 'r') as file:
        inst_name = filename[filename.rfind('/') + 1:]
        
        words = file.read().splitlines()
        
        word_len = len(words[0])
        org_words_num = int(re.search('(?<=\.)[0-9]+', inst_name).group(0))
        org_seq_len = org_words_num + word_len - 1
        neg_errors_number = re.search('(?<=\-)[0-9]+', inst_name)
        neg_errors_number = int(neg_errors_number.group(0)) if neg_errors_number else 0
        pos_errors_number = re.search('(?<=\+)[0-9]+', inst_name)
        pos_errors_number = int(pos_errors_number.group(0)) if pos_errors_number else 0
        
        print("nazwa pliku: " + inst_name)
        print("długość słowa: " + str(word_len))
        print("oryginalna liczba słów: " + str(org_words_num))
        print("oryginalna długość sekwencji: "+ str(org_seq_len))
        print("liczba błędów negatywnych: " + str(neg_errors_number))
        print("liczba błędów pozytywnych: " + str(pos_errors_number))
        print(words[:5])
        print("\n")

<hr />
<h2>Definicja klasy odpowiedzialnej za przechowywanie informacji dla danej instancji problemu.</h2>

In [ ]:
class ProblemPropertiesClass:
    def __init__(self):
        self.words = []
        self.word_len = 0
        self.org_seq_len = 0
        self.words_num = 0
        
        self.common_nucleotides_matrix = np.zeros((self.words_num + 1, self.words_num + 1), dtype=int)
        self.wages = np.zeros(self.word_len)
        self.next_word_power_matrix = np.zeros((self.words_num + 1, self.words_num + 1))
        
        self.avaible_words = np.ones(self.words_num + 1)
        self.avaible_words_number = self.words_num
        self.current_word = 0
        self.current_length = 0
    
    def __init__(self, words, word_len, org_seq_len, words_num):
        self.words = words[:]
        self.word_len = word_len
        self.org_seq_len = org_seq_len
        self.words_num = words_num
        
        self.common_nucleotides_matrix = np.zeros((self.words_num + 1, self.words_num + 1), dtype=int)
        self.wages = np.zeros(self.word_len)
        self.next_word_power_matrix = np.zeros((self.words_num + 1, self.words_num + 1))
        
        self.avaible_words = np.ones(self.words_num + 1)
        self.avaible_words_number = self.words_num
        self.current_word = 0
        self.current_length = 0
        
    
    def prepare_general_vars(self):
        self.common_nucleotides_matrix = np.zeros((self.words_num + 1, self.words_num + 1), dtype=int)
        self.wages = np.zeros(self.word_len)
        self.next_word_power_matrix = np.zeros((self.words_num + 1, self.words_num + 1))
    
    def prepare_attempts_vars(self):
        self.avaible_words = np.ones(self.words_num + 1)    #definicja wektora określającego, czy i-ty wyraz jest dostępny
        self.avaible_words_number = self.words_num           #definicja zmiennej zawierającej liczbę niewykorzystanych jeszcze wyrazów
        self.current_word = 0                                #definicja zmiennej wskazującej ostatnio wybranego słowa -> słowo zerowe, czyli zaczynane jest tworzenie sekwencji
        self.current_length = 0                              #definicja zmiennej zawierającej długość utworzonej dotychczas sekwencji

<hr />
<h2>Wyznaczanie największej ilości znaków, na których sufiks pierwszego wyrazu pokrywa się z prefiksem drugiego wyrazu.</h2>

In [ ]:
#funkcja zwraca maksymalną liczbę nukleotydów, na których zazębia się sufiks nucl1 z prefiksem nucl2
#poszukiwanie poprzez coraz to mniejszą długość ciągu, porównywanie ciągów znak po znaku
def common_nucleotides_max_number_full_iterating(nucl1, nucl2):
    word_len = len(nucl1)
    result = word_len - 1       #zmienna zawierająca końcowy wynik
    
    while(result > 0):          #badanie coraz to mniejszej możliwej długości wspólnego ciągu
        i1 = word_len - result
        i2 = 0
        
        while(i1 < word_len):              #porównanie nachodzących końcówek, znak po znaku
            if(nucl1[i1] != nucl2[i2]):    #znaki się nie zgadzają
                break
            i1 += 1
            i2 += 1
        
        if(i1 == word_len):    #porównywanie ciągów zostało wykonane pozytywnie po wszystkich znakach
            break
        
        result -= 1
    
    return result


#funkcja zwraca maksymalną liczbę nukleotydów, na których zazębia się sufiks nucl1 z prefiksem nucl2
#poszukiwanie poprzez coraz to mniejszą długość ciągu, bezpośrednie porównywanie ciągów przy pomocy mechanizmu "slicing"
def common_nucleotides_max_number_iterating_with_slicing(nucl1, nucl2):
    word_len = len(nucl1)
    result = word_len - 1       #zmienna zawierająca końcowy wynik
    
    while(result > 0):          #badanie coraz to mniejszej możliwej długości wspólnego ciągu
        i1 = word_len - result
        i2 = 0
        
        if(nucl1[-result:] == nucl2[:result]): #porównywanie ciągów przy pomocy mechanizmu "slicing"
            break
        
        result -= 1
    
    return result

In [ ]:
#---------------- Testowanie szybkości wyznaczania wspólego sufiksu nucl1 z prefiksem nucl2 ----------------#
def speed_test_common_nucleotides_max_number(function, repetition):
    print("Function testing in progress...")
    start_time = time.time()
    filename = negative_random_filenames[0]
    with open(filename, 'r') as file:
        inst_name = filename[filename.rfind('/') + 1:]

        words = file.read().splitlines()

        word_len = len(words[0])
        org_words_num = int(re.search('(?<=\.)[0-9]+', inst_name).group(0))
        org_seq_len = org_words_num + word_len - 1
        neg_errors_number = re.search('(?<=\-)[0-9]+', inst_name)
        neg_errors_number = int(neg_errors_number.group(0)) if neg_errors_number else 0
        pos_errors_number = re.search('(?<=\+)[0-9]+', inst_name)
        pos_errors_number = int(pos_errors_number.group(0)) if pos_errors_number else 0
        words_num = len(words)
        
        while(repetition > 0):
            for i in range(words_num):
                for j in range(words_num):
                    if(i != j):
                        number = function(words[i], words[j])
            repetition -= 1
    
    elapsed_time = time.time() - start_time
    print("End of the function testing.")
    print("Time: " + str(elapsed_time) + "\n")

#Wywołanie testowania funkcji
#speed_test_common_nucleotides_max_number(common_nucleotides_max_number_full_iterating, 10)            #repetition=10 Time: 13.98816466331482
#speed_test_common_nucleotides_max_number(common_nucleotides_max_number_iterating_with_slicing, 10)    #repetition=10 Time: 16.22281813621521

<hr />
<h2>Wyznaczanie macierzy zawierającej długości wspólnych sufiksów-prefiksów wszystkich par słów.</h2>

In [ ]:
def generate_common_nucleotides_matrix_full_iterating(problem_properties):
    for i in range(1, problem_properties.words_num + 1):
            for j in range(1, problem_properties.words_num + 1):
                if(i != j):
                    problem_properties.common_nucleotides_matrix[i, j] = common_nucleotides_max_number_full_iterating(problem_properties.words[i], problem_properties.words[j])
                else:
                    problem_properties.common_nucleotides_matrix[i, i] = 0

def generate_common_nucleotides_matrix_with_numpy_tricks_common_nucleotides_max_number(problem_properties, i, j):
    if(i != j):
        return common_nucleotides_max_number_full_iterating(problem_properties.words[i], problem_properties.words[j])
    else:
        return 0

def generate_common_nucleotides_matrix_with_numpy_tricks(problem_properties):
    index_of_column = np.zeros((problem_properties.words_num + 1, problem_properties.words_num + 1), dtype=int)
    index_of_row = np.zeros((problem_properties.words_num + 1, problem_properties.words_num + 1), dtype=int)

    index_of_column[:] = np.arange(problem_properties.words_num + 1)
    index_of_row = index_of_column.transpose()

    vectorized_generate_common_nucleotides_matrix_with_numpy_tricks_common_nucleotides_max_number = np.vectorize(generate_common_nucleotides_matrix_with_numpy_tricks_common_nucleotides_max_number)
    problem_properties.common_nucleotides_matrix[1:, 1:] = vectorized_generate_common_nucleotides_matrix_with_numpy_tricks_common_nucleotides_max_number(problem_properties, index_of_row[1:, 1:], index_of_column[1:, 1:])

In [ ]:
#---------------- Testowanie szybkości wyznaczania macierzy zawierającej długości wspólnych brzegów wszystkich par słów ----------------#
def speed_test_generate_common_nucleotides_matrix(function, repetition):
    print("Preparing required variables...")
    filename = negative_random_filenames[0]
    with open(filename, 'r') as file:
        inst_name = filename[filename.rfind('/') + 1:]

        words = file.read().splitlines()

        word_len = len(words[0])
        org_words_num = int(re.search('(?<=\.)[0-9]+', inst_name).group(0))
        org_seq_len = org_words_num + word_len - 1
        neg_errors_number = re.search('(?<=\-)[0-9]+', inst_name)
        neg_errors_number = int(neg_errors_number.group(0)) if neg_errors_number else 0
        pos_errors_number = re.search('(?<=\+)[0-9]+', inst_name)
        pos_errors_number = int(pos_errors_number.group(0)) if pos_errors_number else 0
        words_num = len(words)

        #dodanie pozornego słowa w celu zaimplementowania sytuacji wyboru pierwszego wyrazu rozpoczynającego sekwencję
        words = ["Poczatek"] + words
        
        
        problem_properties = ProblemPropertiesClass(words, word_len, org_seq_len, words_num)
        
        
        print("Function testing in progress...")
        start_time = time.time()
        while(repetition > 0):
            function(problem_properties)
            repetition -= 1


        elapsed_time = time.time() - start_time
        print("End of the function testing.")
        print("Time: " + str(elapsed_time) + "\n\n")

#Wywołanie testowania funkcji
#speed_test_generate_common_nucleotides_matrix(generate_common_nucleotides_matrix_full_iterating, 10)       #repetition=10 Time: 7.415224313735962
#speed_test_generate_common_nucleotides_matrix(generate_common_nucleotides_matrix_with_numpy_tricks, 10)       #repetition=10 Time: 7.34582257270813

In [ ]:
#---------------- Sprawdzanie poprawności wyznaczania macierzy zawierającej długości wspólnych brzegów wszystkich par słów ----------------#
print("Preparing required variables...")
filename = negative_random_filenames[0]
with open(filename, 'r') as file:
    inst_name = filename[filename.rfind('/') + 1:]

    words = file.read().splitlines()

    word_len = len(words[0])
    org_words_num = int(re.search('(?<=\.)[0-9]+', inst_name).group(0))
    org_seq_len = org_words_num + word_len - 1
    neg_errors_number = re.search('(?<=\-)[0-9]+', inst_name)
    neg_errors_number = int(neg_errors_number.group(0)) if neg_errors_number else 0
    pos_errors_number = re.search('(?<=\+)[0-9]+', inst_name)
    pos_errors_number = int(pos_errors_number.group(0)) if pos_errors_number else 0
    words_num = len(words)

    #dodanie pozornego słowa w celu zaimplementowania sytuacji wyboru pierwszego wyrazu rozpoczynającego sekwencję
    words = ["Poczatek"] + words
    
    
    problem_properties = ProblemPropertiesClass(words, word_len, org_seq_len, words_num)
    
    
    print("Function testing in progress...")
    generate_common_nucleotides_matrix_full_iterating(problem_properties)
    matrix1 = np.copy(problem_properties.common_nucleotides_matrix)
    generate_common_nucleotides_matrix_with_numpy_tricks(problem_properties)
    matrix2 = np.copy(problem_properties.common_nucleotides_matrix)
    
    print(matrix2 - matrix1)
    print(np.sum(matrix2 - matrix1))

    print("End of the function testing.")

<hr />
<h2>Wyznaczanie wektora bazowych wartości wag prawdopodobieństwa, które są obliczane na podstawie długości wspólnego sufiksu-prefiksu wyrazów.</h2>

In [ ]:
def set_starting_wages(n, k):
    result = np.zeros(n)
    n -= 1
    result[n] = 1.0
    while(n > 0):
        n -= 1
        result[n] = result[n + 1] / k
    
    return result

<hr />
<h2>Wyznaczanie wag prawdopodobieństwa dla wyboru j-tego wyrazu po dokonaniu już wyboru i-tego wyrazu.</h2>

In [ ]:
#funkcja oblicza początkowe wagi prawdopodobieństwa dla wyboru j-tego wyrazu po dokonaniu już wyboru i-tego
#waga przyznawana jest na podstawie długości wspólnego sufiksu-prefiksu słowa i-tego i j-tego
def generate_next_word_power_matrix_1_full_iterating(problem_properties):
    #przydzielenia wag dla konkretnych połączeń wyrazów
    for i in range(1, problem_properties.words_num + 1):
        for j in range(1, problem_properties.words_num + 1):
            problem_properties.next_word_power_matrix[i, j] = problem_properties.wages[ problem_properties.common_nucleotides_matrix[i, j] ]
            
            
#funkcja oblicza początkowe wagi prawdopodobieństwa dla wyboru j-tego wyrazu po dokonaniu już wyboru i-tego
#waga przyznawana jest na podstawie długości wspólnego sufiksu-prefiksu słowa i-tego i j-tego
#szybsza wersja wykorzystująca możliwości biblioteki numpy
def generate_next_word_power_matrix_1_with_numpy_tricks(problem_properties):
    #przydzielenia wag dla konkretnych połączeń wyrazów
    problem_properties.next_word_power_matrix[1:, 1:] = problem_properties.wages[ problem_properties.common_nucleotides_matrix[1:, 1:] ]
    #next_word_power_matrix = wages[ common_nucleotides_matrix ]   #ta linia nie modyfikuje globalnie macierzny next_word_power_matrix, będzie trzeba pobawić się w przekazywanie argumentów

#funkcja oblicza początkowe wagi prawdopodobieństwa dla wyboru j-tego wyrazu po dokonaniu już wyboru i-tego
#określona już waga prawdopodobieństwa dla połączenia wyrazów o pewnej długości zazębiających się końcówek jest równomiernie rozdzielana na wsystkie połączenia o tejże długości wspólnego ciągu
def generate_next_word_power_matrix_2_full_iterating(problem_properties):
    for i in range(1, problem_properties.words_num + 1):
        #zliczenie wystąpień zachodzenia wyrazów na danej liczbie znaków
        occurrence_counter = np.zeros(problem_properties.word_len)
        for j in range(1, problem_properties.words_num + 1):
            occurrence_counter[ problem_properties.common_nucleotides_matrix[i, j] ] += 1
        
        #obliczenie wartości wag prawdopodobieństwa obowiązujących dla danego i-tego wiersza -> dla poszukiwania kolejnego wyrazu po i-tym wyrazie
        current_wages = np.copy(problem_properties.wages)
        for j in range(problem_properties.word_len):
            if(occurrence_counter[j] != 0):
                current_wages[j] /= occurrence_counter[j]
        
        #przydzielenie wag dla konkretnych połączeń wyrazów
        for j in range(1, problem_properties.words_num + 1):
            problem_properties.next_word_power_matrix[i, j] = current_wages[ problem_properties.common_nucleotides_matrix[i, j] ]

            
#funkcja oblicza początkowe wagi prawdopodobieństwa dla wyboru j-tego wyrazu po dokonaniu już wyboru i-tego
#określona już waga prawdopodobieństwa dla połączenia wyrazów o pewnej długości zazębiających się końcówek jest równomiernie rozdzielana na wsystkie połączenia o tejże długości wspólnego ciągu
#szybsza wersja wykorzystująca możliwości biblioteki numpy
def generate_next_word_power_matrix_2_with_numpy_tricks_1(problem_properties):
    for i in range(1, problem_properties.words_num + 1):
        #zliczenie wystąpień zachodzenia wyrazów na danej liczbie znaków
        occurrence_counter = np.zeros(problem_properties.word_len)
        unique, counts = np.unique(problem_properties.common_nucleotides_matrix[i, 1:], return_counts=True)
        occurrence_counter[unique] = counts
        
        #obliczenie wartości wag prawdopodobieństwa obowiązujących dla danego i-tego wiersza -> dla poszukiwania kolejnego wyrazu po i-tym wyrazie
        current_wages = np.copy(problem_properties.wages)
        occurrence_counter[occurrence_counter == 0] = 1
        current_wages /= occurrence_counter
        
        #przydzielenia wag dla konkretnych połączeń wyrazów
        problem_properties.next_word_power_matrix[i, 1:] = current_wages[ problem_properties.common_nucleotides_matrix[i, 1:] ]

            
#funkcja oblicza początkowe wagi prawdopodobieństwa dla wyboru j-tego wyrazu po dokonaniu już wyboru i-tego
#określona już waga prawdopodobieństwa dla połączenia wyrazów o pewnej długości zazębiających się końcówek jest równomiernie rozdzielana na wsystkie połączenia o tejże długości wspólnego ciągu
#szybsza wersja wykorzystująca możliwości biblioteki numpy
def generate_next_word_power_matrix_2_with_numpy_tricks_2(problem_properties):
    occurrence_counter = np.zeros((problem_properties.words_num + 1, problem_properties.word_len))
    
    for i in range(1, problem_properties.words_num + 1):
        #zliczenie wystąpień zachodzenia wyrazów na danej liczbie znaków
        unique, counts = np.unique(problem_properties.common_nucleotides_matrix[i, 1:], return_counts=True)
        occurrence_counter[i, unique] = counts
        
    occurrence_counter[occurrence_counter == 0] = 1
        
    #obliczenie wartości wag prawdopodobieństwa obowiązujących dla danego i-tego wiersza -> dla poszukiwania kolejnego wyrazu po i-tym wyrazie
    current_wages = np.zeros((problem_properties.words_num+1, problem_properties.word_len))
    current_wages[:] = problem_properties.wages
    current_wages /= occurrence_counter
    
    #przydzielenia wag dla konkretnych połączeń wyrazów
    for i in range(1, problem_properties.words_num + 1):
        problem_properties.next_word_power_matrix[i, 1:] = current_wages[i, problem_properties.common_nucleotides_matrix[i, 1:] ]

In [ ]:
#---------------- Testowanie szybkości wyznaczania wag prawdopodobień dla wyboru j-tego wyrazu po wybranym już i-tym wyrazie ----------------#
def speed_test_generate_next_word_power_matrix(function, repetition):
    print("Preparing required variables...")
    filename = negative_random_filenames[0]
    with open(filename, 'r') as file:
        inst_name = filename[filename.rfind('/') + 1:]

        words = file.read().splitlines()

        word_len = len(words[0])
        org_words_num = int(re.search('(?<=\.)[0-9]+', inst_name).group(0))
        org_seq_len = org_words_num + word_len - 1
        neg_errors_number = re.search('(?<=\-)[0-9]+', inst_name)
        neg_errors_number = int(neg_errors_number.group(0)) if neg_errors_number else 0
        pos_errors_number = re.search('(?<=\+)[0-9]+', inst_name)
        pos_errors_number = int(pos_errors_number.group(0)) if pos_errors_number else 0
        words_num = len(words)

        #dodanie pozornego słowa w celu zaimplementowania sytuacji wyboru pierwszego wyrazu rozpoczynającego sekwencję
        words = ["Poczatek"] + words
        
        
        problem_properties = ProblemPropertiesClass(words, word_len, org_seq_len, words_num)
        
        
        #wyznaczenie macierzy mówiącej o długości wspólnego i-tego sufiksu z j-tym prefiksem
        generate_common_nucleotides_matrix_full_iterating(problem_properties)

        problem_properties.wages = set_starting_wages(problem_properties.word_len, 2)
        
        
        print("Function testing in progress...")
        start_time = time.time()
        while(repetition > 0):
            function(problem_properties)
            repetition -= 1


        elapsed_time = time.time() - start_time
        print("End of the function testing.")
        print("Time: " + str(elapsed_time) + "\n\n")

#Wywołanie testowania funkcji
#speed_test_generate_next_word_power_matrix(generate_next_word_power_matrix_1_full_iterating, 100)       #repetition=100 Time: 12.014138460159302
#speed_test_generate_next_word_power_matrix(generate_next_word_power_matrix_2_full_iterating, 100)       #repetition=100 Time: 28.95266628265381
#speed_test_generate_next_word_power_matrix(generate_next_word_power_matrix_1_with_numpy_tricks, 100)    #repetition=100 Time: 0.24304628372192383
#speed_test_generate_next_word_power_matrix(generate_next_word_power_matrix_2_with_numpy_tricks_1, 100)  #repetition=100 Time: 2.4654126167297363
#speed_test_generate_next_word_power_matrix(generate_next_word_power_matrix_2_with_numpy_tricks_2, 100)  #repetition=100 Time: 1.8123927116394043

In [ ]:
#---------------- Sprawdzanie poprawności wyznaczania wag prawdopodobień dla wyboru j-tego wyrazu po wybranym już i-tym wyrazie ----------------#
print("Preparing required variables...")
filename = negative_random_filenames[0]
with open(filename, 'r') as file:
    inst_name = filename[filename.rfind('/') + 1:]

    words = file.read().splitlines()

    word_len = len(words[0])
    org_words_num = int(re.search('(?<=\.)[0-9]+', inst_name).group(0))
    org_seq_len = org_words_num + word_len - 1
    neg_errors_number = re.search('(?<=\-)[0-9]+', inst_name)
    neg_errors_number = int(neg_errors_number.group(0)) if neg_errors_number else 0
    pos_errors_number = re.search('(?<=\+)[0-9]+', inst_name)
    pos_errors_number = int(pos_errors_number.group(0)) if pos_errors_number else 0
    words_num = len(words)

    #dodanie pozornego słowa w celu zaimplementowania sytuacji wyboru pierwszego wyrazu rozpoczynającego sekwencję
    words = ["Poczatek"] + words
        
        
    problem_properties = ProblemPropertiesClass(words, word_len, org_seq_len, words_num)


    #wyznaczenie macierzy mówiącej o długości wspólnego i-tego sufiksu z j-tym prefiksem
    generate_common_nucleotides_matrix_full_iterating(problem_properties)

    problem_properties.wages = set_starting_wages(problem_properties.word_len, 2)


    print("Function testing in progress...")
    #generate_next_word_power_matrix_1_full_iterating(problem_properties)
    generate_next_word_power_matrix_2_full_iterating(problem_properties)
    vector1 = np.copy(problem_properties.next_word_power_matrix)
    #generate_next_word_power_matrix_1_with_numpy_tricks(problem_properties)
    #generate_next_word_power_matrix_2_with_numpy_tricks_1(problem_properties)
    generate_next_word_power_matrix_2_with_numpy_tricks_2(problem_properties)
    vector2 = np.copy(problem_properties.next_word_power_matrix)
    
    print(vector2 - vector1)
    print(np.sum(vector2 - vector1))

    print("End of the function testing.")

<hr />
<h2>Wyznaczenie wag prawdopodobieństwa dla wyboru wyrazu rozpoczynającego sekwencje.</h2>

In [ ]:
#funkcja ustawia równe szanse wyboru słowa rozpoczynającego sekwencje
def generate_chance_for_first_word(problem_properties):
    problem_properties.next_word_power_matrix[0, 1:] = 1;

#funkcja ustawia większe szanse wyboru słowa dla wyrazów, których prefiks w maksymalnym przypadku pokrywa się na jaknajmniejszej ilości zanków z jakim kolwiek sufiksem
def generate_chance_for_first_word_1_full_iterating(problem_properties):
    max_common_nucleotides = np.zeros(problem_properties.words_num + 1, dtype=int)

    #wyznaczenie maksymalnej długości zazębiającego się prefiksu j-tego wyrazu
    for j in range(1, problem_properties.words_num + 1):
        max_common_nucleotides[j] = problem_properties.common_nucleotides_matrix[1, j]
        for i in range(2, problem_properties.words_num + 1):
            if(max_common_nucleotides[j] < problem_properties.common_nucleotides_matrix[i, j]):
                max_common_nucleotides[j] = problem_properties.common_nucleotides_matrix[i, j]
    
    #przyznanie wagi prawdopodobieństwa przy użyciu ustalonych wag
    for j in range(1, problem_properties.words_num + 1):
        problem_properties.next_word_power_matrix[0, j] = problem_properties.wages[ problem_properties.word_len - 1 - max_common_nucleotides[j] ]

        
#funkcja ustawia większe szanse wyboru słowa dla wyrazów, których prefiks w maksymalnym przypadku pokrywa się na jaknajmniejszej ilości zanków z jakim kolwiek sufiksem
#szybsza wersja wykorzystująca możliwości biblioteki numpy
def generate_chance_for_first_word_1_with_numpy_tricks(problem_properties):
    #wyznaczenie maksymalnej długości zazębiającego się prefiksu j-tego wyrazu
    max_common_nucleotides = problem_properties.common_nucleotides_matrix.max(0)
    
    #przyznanie wagi prawdopodobieństwa przy użyciu ustalonych wag
    problem_properties.next_word_power_matrix[0] = problem_properties.wages[ problem_properties.word_len - 1 - max_common_nucleotides]
    
    

#funkcja ustawia większe szanse wyboru słowa dla wyrazów, których prefiks w maksymalnym przypadku pokrywa się na jaknajmniejszej ilości zanków z jakim kolwiek sufiksem
#określona już waga jest równomiernie rozdzielona na wyrazy o poszczególnej długości maksymalnego zazębienia
def generate_chance_for_first_word_2_full_iterating(problem_properties):
    max_common_nucleotides = np.zeros(problem_properties.words_num + 1, dtype=int)

    #wyznaczenie maksymalnej długości zazębiającego się prefiksu j-tego wyrazu
    for j in range(1, problem_properties.words_num + 1):
        max_common_nucleotides[j] = problem_properties.common_nucleotides_matrix[1, j]
        for i in range(2, problem_properties.words_num + 1):
            if(max_common_nucleotides[j] < problem_properties.common_nucleotides_matrix[i, j]):
                max_common_nucleotides[j] = problem_properties.common_nucleotides_matrix[i, j]
    
    #przyznanie wagi prawdopodobieństwa przy użyciu ustalonych wag, waga jest równomiernie rozdizelona
    occurrence_counter = np.zeros(problem_properties.word_len)
    for j in range(1, problem_properties.words_num + 1):
        occurrence_counter[ max_common_nucleotides[j] ] += 1

    #obliczenie wartości wag prawdopodobieństwa obowiązujących dla danego i-tego wiersza -> dla poszukiwania kolejnego wyrazu po i-tym wyrazie
    current_wages = np.flip(np.copy(problem_properties.wages))
    for j in range(problem_properties.word_len):
        if(occurrence_counter[j] != 0):
            current_wages[j] /= occurrence_counter[j]

    #przydzielenia wag dla konkretnych połączeń wyrazów
    for j in range(1, problem_properties.words_num + 1):
        problem_properties.next_word_power_matrix[0, j] = current_wages[ max_common_nucleotides[j] ]
        
        
#funkcja ustawia większe szanse wyboru słowa dla wyrazów, których prefiks w maksymalnym przypadku pokrywa się na jaknajmniejszej ilości zanków z jakim kolwiek sufiksem
#określona już waga jest równomiernie rozdzielona na wyrazy o poszczególnej długości maksymalnego zazębienia
#szybsza wersja wykorzystująca możliwości biblioteki numpy
def generate_chance_for_first_word_2_with_numpy_tricks(problem_properties):
    #wyznaczenie maksymalnej długości zazębiającego się prefiksu j-tego wyrazu
    max_common_nucleotides = problem_properties.common_nucleotides_matrix.max(0)
    
    #przyznanie wagi prawdopodobieństwa przy użyciu ustalonych wag, waga jest równomiernie rozdizelona
    occurrence_counter = np.zeros(problem_properties.word_len)
    unique, counts = np.unique(max_common_nucleotides[1:], return_counts=True)
    occurrence_counter[unique] = counts

    #obliczenie wartości wag prawdopodobieństwa obowiązujących dla danego i-tego wiersza -> dla poszukiwania kolejnego wyrazu po i-tym wyrazie
    current_wages = np.flip(np.copy(problem_properties.wages))
    occurrence_counter[occurrence_counter == 0] = 1
    current_wages /= occurrence_counter

    #przydzielenia wag dla konkretnych połączeń wyrazów
    problem_properties.next_word_power_matrix[0, 1:] = current_wages[ max_common_nucleotides[1:]]

In [ ]:
#---------------- Testowanie szybkości wyznaczania wag prawdopodobień dla wyboru pierwszego wyrazu w sekwencji ----------------#
def speed_test_generate_chance_for_first_word(function, repetition):
    print("Preparing required variables...")
    filename = negative_random_filenames[0]
    with open(filename, 'r') as file:
        inst_name = filename[filename.rfind('/') + 1:]

        words = file.read().splitlines()

        word_len = len(words[0])
        org_words_num = int(re.search('(?<=\.)[0-9]+', inst_name).group(0))
        org_seq_len = org_words_num + word_len - 1
        neg_errors_number = re.search('(?<=\-)[0-9]+', inst_name)
        neg_errors_number = int(neg_errors_number.group(0)) if neg_errors_number else 0
        pos_errors_number = re.search('(?<=\+)[0-9]+', inst_name)
        pos_errors_number = int(pos_errors_number.group(0)) if pos_errors_number else 0
        words_num = len(words)

        #dodanie pozornego słowa w celu zaimplementowania sytuacji wyboru pierwszego wyrazu rozpoczynającego sekwencję
        words = ["Poczatek"] + words
        
        
        problem_properties = ProblemPropertiesClass(words, word_len, org_seq_len, words_num)


        #wyznaczenie macierzy mówiącej o długości wspólnego i-tego sufiksu z j-tym prefiksem
        generate_common_nucleotides_matrix_full_iterating(problem_properties)

        problem_properties.wages = set_starting_wages(problem_properties.word_len, 2)
        
        
        print("Function testing in progress...")
        start_time = time.time()
        while(repetition > 0):
            function(problem_properties)
            repetition -= 1


        elapsed_time = time.time() - start_time
        print("End of the function testing.")
        print("Time: " + str(elapsed_time) + "\n\n")

#Wywołanie testowania funkcji
#speed_test_generate_chance_for_first_word(generate_chance_for_first_word, 100)                     #repetition=100 Time: 0.0
#speed_test_generate_chance_for_first_word(generate_chance_for_first_word_1_full_iterating, 100)    #repetition=100 Time: 6.527872323989868
#speed_test_generate_chance_for_first_word(generate_chance_for_first_word_2_full_iterating, 100)    #repetition=100 Time: 6.567286014556885
#speed_test_generate_chance_for_first_word(generate_chance_for_first_word_1_with_numpy_tricks, 100) #repetition=100 Time: 0.017966747283935547
#speed_test_generate_chance_for_first_word(generate_chance_for_first_word_2_with_numpy_tricks, 100) #repetition=100 Time: 0.02245807647705078

In [ ]:
#---------------- Sprawdzanie poprawności wyznaczania wag prawdopodobień dla wyboru pierwszego wyrazu w sekwencji ----------------#
print("Preparing required variables...")
filename = negative_random_filenames[0]
with open(filename, 'r') as file:
    inst_name = filename[filename.rfind('/') + 1:]

    words = file.read().splitlines()

    word_len = len(words[0])
    org_words_num = int(re.search('(?<=\.)[0-9]+', inst_name).group(0))
    org_seq_len = org_words_num + word_len - 1
    neg_errors_number = re.search('(?<=\-)[0-9]+', inst_name)
    neg_errors_number = int(neg_errors_number.group(0)) if neg_errors_number else 0
    pos_errors_number = re.search('(?<=\+)[0-9]+', inst_name)
    pos_errors_number = int(pos_errors_number.group(0)) if pos_errors_number else 0
    words_num = len(words)

    #dodanie pozornego słowa w celu zaimplementowania sytuacji wyboru pierwszego wyrazu rozpoczynającego sekwencję
    words = ["Poczatek"] + words
        
        
    problem_properties = ProblemPropertiesClass(words, word_len, org_seq_len, words_num)


    #wyznaczenie macierzy mówiącej o długości wspólnego i-tego sufiksu z j-tym prefiksem
    generate_common_nucleotides_matrix_full_iterating(problem_properties)

    problem_properties.wages = set_starting_wages(problem_properties.word_len, 2)


    print("Function testing in progress...")
    #generate_chance_for_first_word_1_full_iterating(problem_properties)
    generate_chance_for_first_word_2_full_iterating(problem_properties)
    vector1 = np.copy(problem_properties.next_word_power_matrix[0])
    #generate_chance_for_first_word_1_with_numpy_tricks(problem_properties)
    generate_chance_for_first_word_2_with_numpy_tricks(problem_properties)
    vector2 = np.copy(problem_properties.next_word_power_matrix[0])
    
    print(vector2 - vector1)


    print("End of the function testing.")

<hr />

In [ ]:
#funkcja wybiera kolejny wyraz dla tworzonego ciągu
def choose_next_word(problem_properties):
    #print("avaible words przed: " + str(problem_properties.avaible_words_number))
    #wybranie odpowiednich wektorów z macierzy, które przechowują dane dla ostatnio wybranego słowa
    common_nucleotides_vector = problem_properties.common_nucleotides_matrix[problem_properties.current_word]
    next_word_power_vector = problem_properties.next_word_power_matrix[problem_properties.current_word]
    
    #jeśli obecna długość wynikowego ciągu zwiększona o pełną długość kolejnego słowa przekracza ograniczającą długość ciągu znaków,
    #to trzeba wykluczyć możliwość wyboru kolejnego słowa, które by to za bardzo zwiększyła długość wynikowej sekwencji
    #print(str(problem_properties.current_length) + " + " + str(problem_properties.word_len) + " > " + str(problem_properties.org_seq_len))
    if(problem_properties.current_length + problem_properties.word_len > problem_properties.org_seq_len):
        for j in range(1, problem_properties.words_num + 1):
            if((problem_properties.avaible_words[j]) and (problem_properties.current_length + (problem_properties.word_len - common_nucleotides_vector[j]) > problem_properties.org_seq_len)):
                problem_properties.avaible_words[j] = 0
                problem_properties.avaible_words_number -= 1
        
        #print("Juz prawie koniec - mozna dopasowac jeszcze " + str(problem_properties.avaible_words_number) + " slow")
    
    #nie ma już dostępnych słów do doklejenia
    if(not(problem_properties.avaible_words_number)):
        return -1
    
    #obliczenie sumy mocy prawdopodobieństwa wyboru dla dostępnych słów
    sum_of_probability = 0
    for j in range(1, problem_properties.words_num + 1):
        if(problem_properties.avaible_words[j]):
            sum_of_probability += next_word_power_vector[j]

            
    """#print(str(max_sum) + " < " + str(sum_of_probability))
    if(max_sum < sum_of_probability):
        max_sum = sum_of_probability
        print("sum_of_probability = " + str(sum_of_probability))"""
    
    #realizacja wyboru zgodnie z gęstością prawdopodobieństwa - wybór punktu na odcinku złożonego z sumy mocy prawdopodobieństwa dostępnych połączeń
    point_in_range_of_sum_of_probability = random.uniform(0, sum_of_probability)
    
    #odnalezienie wybranego kolejnego słowa, które zostało wskazane przez wylosowany punkt na odcinku
    j = 1
    while(not(problem_properties.avaible_words[j])):
        j += 1
    
    current_sum_of_probability = next_word_power_vector[j]
    while(point_in_range_of_sum_of_probability > current_sum_of_probability):
        j += 1
        while(not(problem_properties.avaible_words[j])):
            j += 1
        current_sum_of_probability += next_word_power_vector[j]
    
    #print("avaible words po: " + str(problem_properties.avaible_words_number))
    
    return j
    

<hr />
<h2>Przeprowadzanie wygładzania dominujących wartości w macierzy wag prawdopodobieństwa.</h2>

In [ ]:
#funkcja dokonuje wygładzania wartości wag prawdopodobieństw -> zmienia szanse wyboru na bardziej równe, zachowująć jednocześnie porządek preferencji
def compensation_of_next_word_power_matrix_full_iterating(problem_properties):
    compensation_coef = 0.98
    base_of_logarithm = 2
    
    for i in range(problem_properties.words_num + 1):
        max_value = max(problem_properties.next_word_power_matrix[i])
        sum_value = sum(problem_properties.next_word_power_matrix[i])
        
        #jeśli maksymalna wartość zdecydowanie się wyróżnia, to należy złagodzić dysproporcje między wagami prawdopodobieństw
        #print("max_value = " + str(max_value))
        #print("sum_value * compensation_coef = " + str(sum_value * compensation_coef))
        if(max_value > sum_value * compensation_coef):
            #poszukiwanie minimalnej wartości wagi prawdopodobieństwa nie mniejszej niż wartość 1. Wartość ta jest potrzebna do wzoru wygładzającego
            min_value = max_value
            for j in range(problem_properties.words_num + 1):
                if((problem_properties.next_word_power_matrix[i, j] >= 1) and (min_value > problem_properties.next_word_power_matrix[i, j])):
                    min_value = problem_properties.next_word_power_matrix[i, j]
            
            #wygładzanie wag większych niż wartość 1
            for j in range(problem_properties.words_num + 1):
                if(min_value < problem_properties.next_word_power_matrix[i, j]):
                    problem_properties.next_word_power_matrix[i, j] = min_value * (1 + math.log(problem_properties.next_word_power_matrix[i, j] / min_value, base_of_logarithm))

#funkcja dokonuje wygładzania wartości wag prawdopodobieństw -> zmienia szanse wyboru na bardziej równe, zachowująć jednocześnie porządek preferencji
#operacja zmiany wag w danym wierszu została tutaj zwektoryzowana
def compensation_of_next_word_power_matrix_with_numpy_tricks_1(problem_properties):
    compensation_coef = 0.98
    base_of_logarithm = 2
    
    for i in range(problem_properties.words_num + 1):
        max_value = max(problem_properties.next_word_power_matrix[i])
        sum_value = sum(problem_properties.next_word_power_matrix[i])
        
        #jeśli maksymalna wartość zdecydowanie się wyróżnia, to należy złagodzić dysproporcje między wagami prawdopodobieństw
        #print("max_value = " + str(max_value))
        #print("sum_value * compensation_coef = " + str(sum_value * compensation_coef))
        if(max_value > sum_value * compensation_coef):
            #poszukiwanie minimalnej wartości wagi prawdopodobieństwa nie mniejszej niż wartość 1. Wartość ta jest potrzebna do wzoru wygładzającego
            min_value = min(problem_properties.next_word_power_matrix[i, problem_properties.next_word_power_matrix[i] >= 1])
            
            #wygładzanie wag większych niż wartość 1
            problem_properties.next_word_power_matrix[i, problem_properties.next_word_power_matrix[i] > min_value] = min_value * (1 + np.log(problem_properties.next_word_power_matrix[i, problem_properties.next_word_power_matrix[i] > min_value] / min_value) / np.log(base_of_logarithm))

#funkcja dokonuje wygładzania wartości wag prawdopodobieństw -> zmienia szanse wyboru na bardziej równe, zachowująć jednocześnie porządek preferencji
#cała procedura została zwektoryzowana
def compensation_of_next_word_power_matrix_with_numpy_tricks_2(problem_properties):
    compensation_coef = 0.98
    base_of_logarithm = 2
    
    max_value = np.max(problem_properties.next_word_power_matrix, axis=1)
    sum_value = np.sum(problem_properties.next_word_power_matrix, axis=1)
    
    #odwzorowanie maksymalnych wartości wierszy na całą macierz
    max_value_matrix = np.zeros((problem_properties.words_num + 1, problem_properties.words_num + 1))
    max_value_matrix[:] = max_value
    max_value_matrix = max_value_matrix.transpose()

    #odwzorowanie sumy wartości wierszy na całą macierz
    sum_value_matrix = np.zeros((problem_properties.words_num + 1, problem_properties.words_num + 1))
    sum_value_matrix[:] = sum_value
    sum_value_matrix = sum_value_matrix.transpose()

    #wskazanie wierszy w macierzy, w których trzeba wykonać wygładzanie
    mask_row = max_value_matrix > sum_value_matrix * compensation_coef

    #przygotowanie kopii next_word_power_matrix, którą będzie trzeba zmodyfikować w celu odnalezienia minimalnej wartości większej niż 1 dla każdego wiersza
    tmp_next_word_power_matrix = np.copy(problem_properties.next_word_power_matrix)
    mask_tmp = tmp_next_word_power_matrix < 1
    tmp_next_word_power_matrix[mask_tmp] = max_value_matrix[mask_tmp]

    #wyznaczenie i odwzorowanie na całą macierz minimalnej wartości większej niż 1 dla każdego wiersza
    min_value = np.min(tmp_next_word_power_matrix, axis=1)
    min_value_matrix = np.zeros((problem_properties.words_num + 1, problem_properties.words_num + 1))
    min_value_matrix[:] = min_value
    min_value_matrix = min_value_matrix.transpose()

    #wskazanie elementów, które należy zmodyfikować
    mask_final = ((problem_properties.next_word_power_matrix >= min_value_matrix) * (mask_row))

    #wykonanie faktycznego wygładzania dominujących wartości w problem_properties.next_word_power_matrix
    min_value_vector = min_value_matrix[mask_final]
    problem_properties.next_word_power_matrix[mask_final] = min_value_vector * (1 + np.log(problem_properties.next_word_power_matrix[mask_final] / min_value_vector) / np.log(base_of_logarithm))

In [ ]:
#---------------- Testowanie szybkości wygładzania wag ----------------#
def speed_test_compensation_of_next_word_power_matrix(function, repetition, force_compensation):
    print("Preparing required variables...")
    filename = negative_random_filenames[0]
    with open(filename, 'r') as file:
        inst_name = filename[filename.rfind('/') + 1:]

        words = file.read().splitlines()

        word_len = len(words[0])
        org_words_num = int(re.search('(?<=\.)[0-9]+', inst_name).group(0))
        org_seq_len = org_words_num + word_len - 1
        neg_errors_number = re.search('(?<=\-)[0-9]+', inst_name)
        neg_errors_number = int(neg_errors_number.group(0)) if neg_errors_number else 0
        pos_errors_number = re.search('(?<=\+)[0-9]+', inst_name)
        pos_errors_number = int(pos_errors_number.group(0)) if pos_errors_number else 0
        words_num = len(words)

        #dodanie pozornego słowa w celu zaimplementowania sytuacji wyboru pierwszego wyrazu rozpoczynającego sekwencję
        words = ["Poczatek"] + words
        
        
        problem_properties = ProblemPropertiesClass(words, word_len, org_seq_len, words_num)


        #wyznaczenie macierzy mówiącej o długości wspólnego i-tego sufiksu z j-tym prefiksem
        generate_common_nucleotides_matrix_full_iterating(problem_properties)

        problem_properties.wages = set_starting_wages(problem_properties.word_len, 2)


        generate_next_word_power_matrix_1_with_numpy_tricks(problem_properties)
        generate_chance_for_first_word_1_with_numpy_tricks(problem_properties)
        
        print("Function testing in progress...")
        start_time = time.time()
        while(repetition > 0):
            if(force_compensation):
                problem_properties.next_word_power_matrix[:, 2] = 1000000
            function(problem_properties)
            repetition -= 1


        elapsed_time = time.time() - start_time
        print("End of the function testing.")
        print("Time: " + str(elapsed_time) + "\n\n")

#Wywołanie testowania funkcji
#speed_test_compensation_of_next_word_power_matrix(compensation_of_next_word_power_matrix_full_iterating, 10, force_compensation=False)       #repetition=10 Time: 0.30295610427856445
#speed_test_compensation_of_next_word_power_matrix(compensation_of_next_word_power_matrix_full_iterating, 10, force_compensation=True)        #repetition=10 Time: 1.3494906425476074
#speed_test_compensation_of_next_word_power_matrix(compensation_of_next_word_power_matrix_with_numpy_tricks_1, 10, force_compensation=False)  #repetition=10 Time: 0.3110346794128418
#speed_test_compensation_of_next_word_power_matrix(compensation_of_next_word_power_matrix_with_numpy_tricks_1, 10, force_compensation=True)   #repetition=10 Time: 0.45663905143737793
#speed_test_compensation_of_next_word_power_matrix(compensation_of_next_word_power_matrix_with_numpy_tricks_2, 10, force_compensation=False)  #repetition=10 Time: 0.08384537696838379
#speed_test_compensation_of_next_word_power_matrix(compensation_of_next_word_power_matrix_with_numpy_tricks_2, 10, force_compensation=True)   #repetition=10 Time: 0.08833527565002441

In [ ]:
#---------------- Sprawdzanie poprawności wygładzania wag ----------------#
print("Preparing required variables...")
filename = negative_random_filenames[0]
with open(filename, 'r') as file:
    inst_name = filename[filename.rfind('/') + 1:]

    words = file.read().splitlines()

    word_len = len(words[0])
    org_words_num = int(re.search('(?<=\.)[0-9]+', inst_name).group(0))
    org_seq_len = org_words_num + word_len - 1
    neg_errors_number = re.search('(?<=\-)[0-9]+', inst_name)
    neg_errors_number = int(neg_errors_number.group(0)) if neg_errors_number else 0
    pos_errors_number = re.search('(?<=\+)[0-9]+', inst_name)
    pos_errors_number = int(pos_errors_number.group(0)) if pos_errors_number else 0
    words_num = len(words)

    #dodanie pozornego słowa w celu zaimplementowania sytuacji wyboru pierwszego wyrazu rozpoczynającego sekwencję
    words = ["Poczatek"] + words
        
        
    problem_properties1 = ProblemPropertiesClass(words, word_len, org_seq_len, words_num)
    problem_properties2 = ProblemPropertiesClass(words, word_len, org_seq_len, words_num)
    


    #wyznaczenie macierzy mówiącej o długości wspólnego i-tego sufiksu z j-tym prefiksem
    generate_common_nucleotides_matrix_full_iterating(problem_properties1)
    generate_common_nucleotides_matrix_full_iterating(problem_properties2)

    problem_properties1.wages = set_starting_wages(problem_properties1.word_len, 2)
    problem_properties2.wages = set_starting_wages(problem_properties2.word_len, 2)


    generate_next_word_power_matrix_1_with_numpy_tricks(problem_properties1)
    generate_chance_for_first_word_2_with_numpy_tricks(problem_properties1)
    problem_properties1.next_word_power_matrix[:, 2] = 1000000
    
    generate_next_word_power_matrix_1_with_numpy_tricks(problem_properties2)
    generate_chance_for_first_word_2_with_numpy_tricks(problem_properties2)
    problem_properties2.next_word_power_matrix[:, 2] = 1000000
    
    
    print("Function testing in progress...")
    compensation_of_next_word_power_matrix_full_iterating(problem_properties1)
    matrix1 = np.copy(problem_properties1.next_word_power_matrix)
    print(problem_properties1.next_word_power_matrix)
    
    #compensation_of_next_word_power_matrix_with_numpy_tricks_1(problem_properties2)
    compensation_of_next_word_power_matrix_with_numpy_tricks_2(problem_properties2)
    matrix2 = np.copy(problem_properties2.next_word_power_matrix)
    print(problem_properties2.next_word_power_matrix)
    
    print(matrix2 - matrix1)
    print("sum: " + str(np.sum(matrix2 - matrix1)))


    print("End of the function testing.")

<hr/>
<h1>Główny kod wykonujący obliczenia zgodnie z algorytmem mrówkowym</h1>

In [ ]:
random.seed(0)
filename = negative_random_filenames[0]
with open(filename, 'r') as file:
    inst_name = filename[filename.rfind('/') + 1:]

    words = file.read().splitlines()

    word_len = len(words[0])
    org_words_num = int(re.search('(?<=\.)[0-9]+', inst_name).group(0))
    org_seq_len = org_words_num + word_len - 1
    neg_errors_number = re.search('(?<=\-)[0-9]+', inst_name)
    neg_errors_number = int(neg_errors_number.group(0)) if neg_errors_number else 0
    pos_errors_number = re.search('(?<=\+)[0-9]+', inst_name)
    pos_errors_number = int(pos_errors_number.group(0)) if pos_errors_number else 0
    words_num = len(words)

    print("nazwa pliku: " + inst_name)
    print("długość słowa: " + str(word_len))
    print("oryginalna liczba słów: " + str(org_words_num))
    print("oryginalna długość sekwencji: "+ str(org_seq_len))
    print("liczba błędów negatywnych: " + str(neg_errors_number))
    print("liczba błędów pozytywnych: " + str(pos_errors_number))
    print("liczba oligonukleotydów: " + str(words_num))
    
    #dodanie pozornego słowa w celu zaimplementowania sytuacji wyboru pierwszego wyrazu rozpoczynającego sekwencję
    words = ["Poczatek"] + words
    print(words[:5])
    print("\n")
    
    
    #zapisanie ważnych informacji w obiekcie klasy ProblemPropertiesClass
    problem_properties = ProblemPropertiesClass(words, word_len, org_seq_len, words_num)
    
    
    #wyznaczenie macierzy mówiącej o długości wspólnego i-tego sufiksu z j-tym prefiksem
    print("common_nucleotides_matrix")
    generate_common_nucleotides_matrix_full_iterating(problem_properties)
    #print(common_nucleotides_matrix[0, :])
    
    print("set_starting_wages()")
    problem_properties.wages = set_starting_wages(problem_properties.word_len, 2)
    #print(wages)
    
    
    #wyznaczanie macierzy mocy prawdopodobieństwa następienia j-tego wyrazu po i-tym
    #generate_next_word_power_matrix_1_full_iterating(problem_properties)
    #generate_next_word_power_matrix_2_full_iterating(problem_properties)
    #print("generate_next_word_power_matrix_1_with_numpy_tricks()")
    #generate_next_word_power_matrix_1_with_numpy_tricks(problem_properties)
    #print("generate_next_word_power_matrix_2_with_numpy_tricks_1()")
    #generate_next_word_power_matrix_2_with_numpy_tricks_1(problem_properties)
    print("generate_next_word_power_matrix_2_with_numpy_tricks_2()")
    generate_next_word_power_matrix_2_with_numpy_tricks_2(problem_properties)
    
    #ustawienie wag prawdopodobieństwa dla rozpoczęcia tworzenia sekwencji przez dany wyraz
    #print("generate_chance_for_first_word()")
    #generate_chance_for_first_word(problem_properties)
    #print("generate_chance_for_first_word_1_with_numpy_tricks()")
    #generate_chance_for_first_word_1_with_numpy_tricks(problem_properties)
    print("generate_chance_for_first_word_2_with_numpy_tricks()")
    generate_chance_for_first_word_2_with_numpy_tricks(problem_properties)
    
    
    iterations_number = 100    #definicja zmiennej określającej ile iteracji będzie trwał algorytm mrówkowy
    #max_w_macierzy = 0
    
    #wykonywanie poszczególnej iteracji; wykonywanie pętli tyle razy, ile jest zadeklarowanych iteracji do wykonania
    for indeks_of_iteration in range(iterations_number):
        attempts_number = 100  #definicja zmiennej określającej ile razy dokonywane jest budowanie sekwencji w jednej iteracji
        sequences = []
        
        
        #tworzenie wynikowej sekwencji; powtarzanie prób uzyskania rozwiązania tyle razy, ile jest to określone parametrem 'attempts_number'
        print("\nIteracja " + str(indeks_of_iteration) + ":\t\nTrwa wykonywanie wszystkich " + str(attempts_number) + " prób utworzenia wynikowej sekwencji.")
        for indeks_of_attempt in range(attempts_number):
            problem_properties.prepare_attempts_vars()
            sequence = []
            
            problem_properties.avaible_words[0] = 0        #odznaczenie użycia już zerowego słowa ('X')
            problem_properties.current_word = 0            #definicja zmiennej wskazującej ostatnio wybranego słowa -> słowo zerowe, czyli zaczynane jest tworzenie sekwencji
            sequence.append(problem_properties.current_word)
            
            #nieustannie wykonująca się pętla - zostanie przerwana, gdy nie będzie już możliwości doczepienia słowa przy ograniczeniu ilości znaków w wynikowym ciągu
            while(True):
                next_word = choose_next_word(problem_properties)
                #print("Wybrane slowo: " + str(next_word))
                
                if(next_word == -1):
                    break
                
                problem_properties.current_length += problem_properties.word_len - problem_properties.common_nucleotides_matrix[problem_properties.current_word, next_word]
                problem_properties.avaible_words_number -= 1
                problem_properties.avaible_words[next_word]= 0
                
                """print(problem_properties.words[problem_properties.current_word])
                print((problem_properties.word_len - problem_properties.common_nucleotides_matrix[problem_properties.current_word, next_word]) * ' ' + problem_properties.words[problem_properties.next_word] + "\tPokrywanie na tylu znakach: " + str(problem_properties.common_nucleotides_matrix[problem_properties.current_word, next_word]))
                print("Liczba niewykorzystanych jeszcze slow: " + str(problem_properties.avaible_words_number))
                print("Obecna dlugosc ciagu: " + str(problem_properties.current_length))
                print("\n")"""
                
                problem_properties.current_word = next_word
                sequence.append(problem_properties.current_word)
            
            #print("Koniec tworzenia ciagu.")
            sequences.append(sequence)
            
        print("Wykonano wszystkie sekwencje w obecnej iteracji")
        """for i in range(len(sequences)):
            print(str(i) + ":\t" + str(len(sequences[i])))"""
            
        sequences.sort(key = lambda x: len(x), reverse=True)

        """for i in range(len(sequences)):
            print(str(i) + ":\t" + str(len(sequences[i])))"""
        
        print(sequences[0])
        #print(sequences[1])
        #nagrodzenie najlepszych rozwiązań
        part_of_the_best_sequences = 0.1
        amount_of_the_best_sequences = int(attempts_number * part_of_the_best_sequences)
        for i in range(amount_of_the_best_sequences):
            print(str(i) + ":\t" + str(len(sequences[i])))
        best_price_for_sequence = 1.2
        for iSequence in range(amount_of_the_best_sequences):
            sequence = sequences[iSequence]
            price_for_sequence = 1 + (best_price_for_sequence - 1) * ((amount_of_the_best_sequences - iSequence) / amount_of_the_best_sequences)
            #print("price_for_sequence = " + str(price_for_sequence))
            for iConnection in range(len(sequence) - 1):
                first_word = sequence[iConnection]
                second_word = sequence[iConnection + 1]
                
                price_for_connection = 1 + (price_for_sequence - 1) * (problem_properties.common_nucleotides_matrix[first_word, second_word] / (problem_properties.word_len - 1))
                problem_properties.next_word_power_matrix[first_word, second_word] *= price_for_connection
                
                """#print(str(max_sum) + " < " + str(sum_of_probability))
                if(max_w_macierzy < problem_properties.next_word_power_matrix[first_word, second_word]):
                    max_w_macierzy = problem_properties.next_word_power_matrix[first_word, second_word]
                    print("max_w_macierzy = " + str(max_w_macierzy))
                    print("nagradzalem mnozac przez " + str(price_for_connection))
                    print("price_for_sequence = " + str(price_for_sequence))"""
            
            
            """wyswietlanie mnożnika dla połączeń - im bardziej wyrazy się zazębiają, tym większa liniowo jest wartość mnożnika
            for i in range(problem_properties.word_len):
                price_for_connection = 1 + (price_for_sequence - 1) * (i / (problem_properties.word_len - 1))
                print("price_for_connection = " + str(price_for_connection))"""
    
    
        #wygładzanie macierzy wag prawdopodobieństwa w celu zapobiegnięcia nadmiernemu zdominowaniu przez najlepsze rozwiązanie
        print("wykonywanie funkcji compensation_of_next_word_power_matrix...")
        compensation_of_next_word_power_matrix_with_numpy_tricks_2(problem_properties)
    
    print("koniec")

<hr /><hr /><br /><br /><br />
<h3>Poniżej można swobodnie testować i sprawdzać działanie wykorzystywanych konstrukcji języka python.</h3>

In [ ]:
avaible_words_number = 0
def testF():
    print(testVar)
    print("testVar2 = " + str(testVar2))

In [ ]:
testVar = 5
testF()
for i in range(5):
    testVar2 = i
    testF()

In [ ]:
for i in range(20):
    print(random.uniform(0.9992892359651135, 0.9992892359651136))

In [ ]:
def testPrzekazywanie(x):
    x += 10

x = 5
print(x)
testPrzekazywanie(x)
print(x)

In [ ]:
vartest = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]])
print(vartest)
print(vartest.shape)
print(vartest[1])
print(max(vartest[1]))

In [ ]:
print(math.log(625, 5))

In [ ]:
x = np.zeros(6)
print(x)
y = np.zeros((4))
print(y)
z = np.zeros(13, dtype=int)
print(z)

In [ ]:
"""#wyznaczenie maksymalnej długości zazębiającego się prefiksu j-tego wyrazu
for j in range(1, words_num + 1):
    max_common_nucleotides[j] = common_nucleotides_matrix[1, j]
    for i in range(2, words_num + 1):
        if(max_common_nucleotides[j] < common_nucleotides_matrix[i, j]):
            max_common_nucleotides[j] = common_nucleotides_matrix[i, j]"""

x = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(x)
y = x.max(0)
print(y)



In [ ]:
"""#przyznanie wagi prawdopodobieństwa przy użyciu ustalonych wag
    for j in range(1, words_num + 1):
        next_word_power_matrix[0, j] = wages[ word_len - 1 - max_common_nucleotides[j] ]"""


a = np.array([1, 2, 3, 5, 6])
print(a)
b = np.array([0, 1, 2, 1, 0])
print(b)
values = np.array([5, 20, 100])
print(values)

a = values[2 - b]
print(a)


In [ ]:
"""occurrence_counter = np.zeros(word_len)
for j in range(1, words_num + 1):
    occurrence_counter[ common_nucleotides_matrix[0, j] ] += 1"""

x = np.array([0, 1,3, 2, 1, 1, 1, 4, 6, 6, 2])
y = np.zeros(7, dtype=int)
unique, counts = np.unique(x, return_counts=True)
y[unique] = counts
print(y)


In [ ]:
"""#obliczenie wartości wag prawdopodobieństwa obowiązujących dla danego i-tego wiersza -> dla poszukiwania kolejnego wyrazu po i-tym wyrazie
    current_wages = np.copy(wages)
    for j in range(word_len):
        if(occurrence_counter[j] != 0):
            current_wages[j] /= occurrence_counter[j]"""

x = np.array([1, 4, 9, 4, 10, 30])
y = np.array([1, 2, 3, 0, 2, 5])

print(y)
y[y == 0] = 1
print(y)

z = x / y
#z = np.where(y != 0, x / y, x)
#z = np.where(y != 0, 'a', 'b')
print(z)

In [ ]:
"""#funkcja oblicza początkowe wagi prawdopodobieństwa dla wyboru j-tego wyrazu po dokonaniu już wyboru i-tego
#waga przyznawana jest na podstawie długości wspólnego sufiksu-prefiksu słowa i-tego i j-tego
#szybsza wersja wykorzystująca możliwości biblioteki numpy
def generate_next_word_power_matrix_1_with_numpy_tricks():
    #przydzielenia wag dla konkretnych połączeń wyrazów
    for i in range(1, words_num + 1):
        for j in range(1, words_num + 1):
            next_word_power_matrix[i, j] = wages[ common_nucleotides_matrix[i, j] ]"""

x = np.zeros((3, 3))
print(x)
y = np.array([10, 20, 30])
print(y)
z = np.array([[0, 1, 2], [1, 1, 1], [2, 2, 0]])
print(z)
x = y[z]
print(x)

In [ ]:
"""#funkcja oblicza początkowe wagi prawdopodobieństwa dla wyboru j-tego wyrazu po dokonaniu już wyboru i-tego
#określona już waga prawdopodobieństwa dla połączenia wyrazów o pewnej długości zazębiających się końcówek jest równomiernie rozdzielana na wsystkie połączenia o tejże długości wspólnego ciągu
#szybsza wersja wykorzystująca możliwości biblioteki numpy
def generate_next_word_power_matrix_2_with_numpy_tricks2():
    for i in range(1, words_num + 1):
        #zliczenie wystąpień zachodzenia wyrazów na danej liczbie znaków
        occurrence_counter = np.zeros(word_len)
        unique, counts = np.unique(common_nucleotides_matrix[i, 1:], return_counts=True)
        occurrence_counter[unique] = counts
        
        #obliczenie wartości wag prawdopodobieństwa obowiązujących dla danego i-tego wiersza -> dla poszukiwania kolejnego wyrazu po i-tym wyrazie
        current_wages = np.copy(wages)
        occurrence_counter[occurrence_counter == 0] = 1
        current_wages /= occurrence_counter
        
        #przydzielenia wag dla konkretnych połączeń wyrazów
        next_word_power_matrix[i] = current_wages[ common_nucleotides_matrix[i] ]"""

def my_func(a):
    print("a = " + str(a))
    return np.unique(a, return_counts=True)

x = np.array([[0, 1, 2, 4, 3], [1, 2, 0, 3, 4], [1, 2, 3, 4, 4]])
print(x)
result = np.apply_along_axis(my_func, axis=1, arr=x)
#unique, counts = np.unique(x, return_counts=True, axis=0)
print(result)
#print(unique)
#print(counts)

In [ ]:
a = [1, 2, 3, 4, 5]
b = [1, 0, 0, 1, 1]
print(a[b])

In [ ]:
x = np.zeros((7, 4))
ind = [0, 2, 3]
for i in range(7):
    x[i, ind] = 999

print(x)

In [ ]:
"""current_wages = np.copy(wages)
current_wages /= occurrence_counter"""

x = np.zeros((7, 4))
y = [0, 2, 3, 10]
x[:] = y
y[1] = -10

print(x)
print(y)

In [ ]:
x = np.arange(5)
print(x)

In [ ]:
"""next_word_power_matrix[:,:] = current_wages[ind, common_nucleotides_matrix[ind] ]"""

ind = np.arange(3)
print(ind)

x = np.zeros((3, 3))
w = np.array([[10, 20, 30, 40, 50], [100, 200, 300, 400, 500], [1000, 2000, 3000, 4000, 5000]])
print(w)

c = np.array([[0, 1, 2], [2, 3, 4], [0, 2, 4]])

tmp = (0, [0, 1, 2])
#x[:, :] = w[ind, c[ind]]     #to jest źle
x[:, :] = w[tmp]


"""x[:, :] = w[ind, c]
for i in range(3):
    x[i] = w[i, c[i]]"""

print(x)

In [ ]:
"""def generate_common_nucleotides_matrix_full_iterating():
    for i in range(1, words_num + 1):
            for j in range(1, words_num + 1):
                if(i != j):
                    common_nucleotides_matrix[i, j] = common_nucleotides_max_number_full_iterating(words[i], words[j])
                else:
                    common_nucleotides_matrix[i, i] = 0"""

def testFunc(a, b):
    result = 0
    if(a < b):
        result = a + 10
    else:
        result = b - 100
    return result

x = np.zeros((5, 5))
y = np.zeros((5, 5))
ind = np.arange(5)

x[:] = ind
print(x)


y = x.transpose()
print(y)

vTestFunc = np.vectorize(testFunc)
result = vTestFunc(x, y)
print(result)


In [ ]:
"""min_value = max_value
for j in range(1, problem_properties.words_num + 1):
    if((problem_properties.next_word_power_matrix[i, j] >= 1) and (min_value > problem_properties.next_word_power_matrix[i, j])):
        min_value = problem_properties.next_word_power_matrix[i, j]"""

x = np.array([[2, 3.4, 3, 5, 0.3, -4, 2, 1.2, 1.6, 0.3, -0.1, 0.9],
              [4, 3.2, 2, 6, 0.6, -3, 6, 1.7, 1.1, 0.2, -0.5, 0.8]])
result = min(x[0, x[0] >= 1])
print(result)

In [ ]:
"""for j in range(1, problem_properties.words_num + 1):
    if(min_value < problem_properties.next_word_power_matrix[i, j]):
        problem_properties.next_word_power_matrix[i, j] = min_value * (1 + math.log(problem_properties.next_word_power_matrix[i, j] / min_value, base_of_logarithm))"""

x = np.array([[0.9, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
              [-4, -2, 0, 2, 4, 6, 8, 10, 12, 14, 16]])
print(x)

min_value = 5
#x[0, x[0] > min_value] = 2 * (1 + x[0, x[0] > min_value])
tmp = x[0, x[0] > min_value]
x[0, x[0] > min_value] = 2 * (1 + x[0, x[0] > min_value])
print(x[0])
print(np.log(x[0]) / np.log(2))

In [ ]:
x = np.array([1, 2, 3, 4, 5])
print(x)
y = np.flip(x)
y[3] = 999
print(x)
print(y)

In [ ]:
x = np.array([1, 2, 3, 4, 5])
print(x)
y = x[::-1]
y[3] = 999
print(x)
print(y)

In [ ]:
"""for i in range(problem_properties.words_num + 1):
    max_value = max(problem_properties.next_word_power_matrix[i])
    sum_value = sum(problem_properties.next_word_power_matrix[i])"""

x = np.array([[1, 2, 5], 
              [4, 7, 3]])

max_value = np.max(x, axis=1)
print(max_value)

sum_value = np.sum(x, axis=1)
print(sum_value)

print(max_value > sum_value * 0.6)

In [ ]:
"""for i in range(problem_properties.words_num + 1):
    if(max_value > sum_value * compensation_coef):
        #poszukiwanie minimalnej wartości wagi prawdopodobieństwa nie mniejszej niż wartość 1. Wartość ta jest potrzebna do wzoru wygładzającego
        min_value = min(problem_properties.next_word_power_matrix[i, problem_properties.next_word_power_matrix[i] >= 1])

        #wygładzanie wag większych niż wartość 1
        problem_properties.next_word_power_matrix[i, problem_properties.next_word_power_matrix[i] > min_value] = min_value * (1 + np.log(problem_properties.next_word_power_matrix[i, problem_properties.next_word_power_matrix[i] > min_value] / min_value) / np.log(base_of_logarithm))"""

x = np.array([[1, 2, 5], 
              [4, 7, 3], 
              [1, 10, 4], 
              [6, 7, 8]])

max_value = np.max(x, axis=1)
print(max_value)
max_value_matrix = np.zeros((3, 4))
max_value_matrix[:] = max_value
max_value_matrix = max_value_matrix.transpose()
print(max_value_matrix)

sum_value = np.sum(x, axis=1)
print(sum_value)
sum_value_matrix = np.zeros((3, 4))
sum_value_matrix[:] = sum_value
sum_value_matrix = sum_value_matrix.transpose()
print(sum_value_matrix)

mask_row = max_value_matrix > sum_value_matrix * 0.6
print(mask_row)

tmpx = np.copy(x)
print("\ntmpx przed:\n" + str(tmpx))
wsk = tmpx < 3
tmpx[wsk] = max_value_matrix[wsk]
print("\ntmpx po:\n" + str(tmpx))

min_value = np.min(tmpx, axis=1)
print(min_value)
min_value_matrix = np.zeros((3, 4))
min_value_matrix[:] = min_value
min_value_matrix = min_value_matrix.transpose()
print(min_value_matrix)

mask_final = ((x >= min_value_matrix) * (mask_row))
print(mask_final)

print("\nx przed:\n" + str(x))
x[mask_final] = min_value_matrix[mask_final] * (1 + x[mask_final])
print("\nx po:\n" + str(x))

